In [4]:
import os
import torch
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
from datasetRegression import EEGDataset # Get the dataset class specific for regression models
from torch.utils.data import random_split


In [5]:
# load in the dataset

#raw_data_dir = '//uni.au.dk/dfs/Tech_EarEEG/Students/RD2022_Artefact_AkselStark/data/1A/study_1A_mat_simple'
raw_data_dir = '../data'

trainingNights = 7
testNights = 2

print("Test set")
ds2 = EEGDataset(raw_data_dir,testNights, 250, skips = trainingNights)

print("Training set")
ds1 = EEGDataset(raw_data_dir,trainingNights, 250, skips = 0) #Instantiate a dataset using the directory of data, amount of night to include and amount of samples in a segment



Test set
../data/study_1A_mat_simple/S_01/night_1/artefact_annotations.npy
../data/study_1A_mat_simple/S_01/night_1/EEG_raw_250hz.npy
../data/study_1A_mat_simple/S_01/night_2/artefact_annotations.npy
../data/study_1A_mat_simple/S_01/night_2/EEG_raw_250hz.npy
../data/study_1A_mat_simple/S_01/night_3/artefact_annotations.npy
../data/study_1A_mat_simple/S_01/night_3/EEG_raw_250hz.npy
../data/study_1A_mat_simple/S_01/night_4/artefact_annotations.npy
../data/study_1A_mat_simple/S_01/night_4/EEG_raw_250hz.npy
../data/study_1A_mat_simple/S_02/night_1/artefact_annotations.npy
../data/study_1A_mat_simple/S_02/night_1/EEG_raw_250hz.npy
../data/study_1A_mat_simple/S_02/night_2/artefact_annotations.npy
../data/study_1A_mat_simple/S_02/night_2/EEG_raw_250hz.npy
../data/study_1A_mat_simple/S_02/night_3/artefact_annotations.npy
../data/study_1A_mat_simple/S_02/night_3/EEG_raw_250hz.npy
../data/study_1A_mat_simple/S_02/night_4/artefact_annotations.npy
Lables for night 0 loaded
../data/study_1A_mat_sim

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' #Check for cuda 
print(f'Using {device} device')

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        #self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.BatchNorm1d(250),
            nn.Linear(250, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
            nn.Sigmoid(), 
        )

    def forward(self, x):
        #x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

Using cuda device


In [7]:
learning_rate = 1e-3
batch_size = 64


model = NeuralNetwork()
model.to(device)

loss = nn.MSELoss() #L2 loss

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)
        # Compute prediction and loss
        pred = model(X)# Reshape to 1 dimension if using binary classification, otherwise keep dimensions from model output
        pred = pred.reshape(-1)
        pred = pred.to(device)
        yFloat = y.type(torch.FloatTensor).to(device)
        
        loss = loss_fn(pred, yFloat)
        


        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 1000 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            #print("Predicted values:")
            #print(pred)
            #print("Actual values:")
            #print(yFloat)



def test_loop(dataloader_test, model, loss_fn, test_set = True):
    size = len(dataloader_test.dataset)
    num_batches = len(dataloader_test)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader_test:
            X = X.to(device)
            y = y.to(device)
            pred = model(X).reshape(-1).to(device) # Reshape to 1 dimension if using binary classification, otherwise keep dimensions from model output
            test_loss += loss_fn(pred, y.type(torch.FloatTensor).to(device)).item()
            #correct += (pred.round() == y).type(torch.float).sum().item()
            

    test_loss /= num_batches
    #correct /= size
    if test_set:
        #print(f"Test set Error: \n Test Set Accuracy: {(100*correct):>0.5f}%, Avg Test Set loss: {test_loss:>8f} \n")
        print(f"Test set Error: \n Avg Test Set loss: {test_loss:>8f} \n")
    else:
        #print(f"Training Set Error: \n Training Set Accuracy: {(100*correct):>0.5f}%, Avg Training Set loss: {test_loss:>8f} \n")
        print(f"Training Set Error: \n Avg Training Set loss: {test_loss:>8f} \n")


In [9]:
# Split data into train and test data (Commented out: Changed to loading different dataset class instances)
#trainSamples = int(ds1.__len__()*0.7)
#testSamples = int(ds1.__len__() - trainSamples)
#training_data, test_data = random_split(ds1, (trainSamples,testSamples), generator=torch.Generator().manual_seed(42))

#train_dataloader = DataLoader(training_data, batch_size=64, drop_last = True) # Drop_last, to avoid incomplete batches, which won't work with weighted loss
#test_dataloader = DataLoader(test_data, batch_size=64, drop_last = True) # Drop_last, to avoid incomplete batches, which won't work with weighted loss


train_dataloader = DataLoader(ds1, batch_size=64, drop_last = True) # Drop_last, to avoid incomplete batches, which won't work with weighted loss
test_dataloader = DataLoader(ds2, batch_size=64, drop_last = True) # Drop_last, to avoid incomplete batches, which won't work with weighted loss



In [10]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    
    train_loop(train_dataloader, model, loss, optimizer)    
    test_loop(test_dataloader, model, loss)
    test_loop(train_dataloader, model, loss, test_set = False)

print("Done!")

Epoch 1
-------------------------------
loss: 0.255178  [    0/4951495]
loss: 0.019560  [64000/4951495]
loss: 0.105738  [128000/4951495]
loss: 0.504565  [192000/4951495]
loss: 0.016787  [256000/4951495]
loss: 0.501681  [320000/4951495]
loss: 0.503782  [384000/4951495]
loss: 0.495368  [448000/4951495]
loss: 0.060822  [512000/4951495]
loss: 0.500505  [576000/4951495]
loss: 0.501385  [640000/4951495]
loss: 0.500000  [704000/4951495]
loss: 0.500000  [768000/4951495]
loss: 0.503915  [832000/4951495]
loss: 0.505674  [896000/4951495]
loss: 0.500870  [960000/4951495]
loss: 0.502694  [1024000/4951495]
loss: 0.500000  [1088000/4951495]
loss: 0.500000  [1152000/4951495]
loss: 0.419200  [1216000/4951495]
loss: 0.506871  [1280000/4951495]
loss: 0.500000  [1344000/4951495]
loss: 0.500730  [1408000/4951495]
loss: 0.503165  [1472000/4951495]
loss: 0.500268  [1536000/4951495]
loss: 0.503625  [1600000/4951495]
loss: 0.503538  [1664000/4951495]
loss: 0.500272  [1728000/4951495]
loss: 0.503169  [1792000/4